In [3]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'-1'

In [4]:
import argparse
import gc
import math
import os
from argparse import Namespace
from datetime import timedelta
from multiprocessing import cpu_count
from typing import Any, List, Optional, Tuple, Union, Dict


import IPython
import joblib
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import transformers
import wandb
from fastai import *
from fastai.basics import *
from fastai.callbacks import *
from fastai.metrics import *
from fastai.metrics import CMScores
from fastai.text import *
from fastai.utils.ipython import *
from fastprogress.fastprogress import progress_bar
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             classification_report, confusion_matrix, f1_score,
                             matthews_corrcoef)
from sklearn.preprocessing import OrdinalEncoder
from torch import nn as nn
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from torchmetrics import Metric
from transformers import (AdamW, AutoConfig, AutoModel, AutoModelWithLMHead,
                          AutoTokenizer, BertConfig, BertModel, BertTokenizer,
                          BigBirdTokenizer, get_linear_schedule_with_warmup)



%matplotlib inline

In [5]:
@np_func
def mcc(inp,targ): return matthews_corrcoef(targ, np.argmax(inp, axis=-1))

In [4]:
df_test = pd.read_parquet('../data/test_en.parquet')
df_humans = pd.read_parquet('../data/humans_en.parquet')

In [5]:
defaults.device = torch.device('cuda:0')

# ULMFiT forward

In [6]:
# learn_test_fwd = load_learner(os.getcwd(), 'saved_models_ulmfit/ulmfit_forward_exported', backwards=False)
# learn_humans_fwd = load_learner(os.getcwd(), 'saved_models_ulmfit/ulmfit_forward_exported', backwards=False)
ulmfit_fwd = load_learner(os.getcwd(), 'saved_models_ulmfit/ulmfit_forward_exported', backwards=False)

In [7]:
MODEL_NAME = 'ULMFiT-fwd'

DS_TYPE = 'humans'
DS = df_humans if DS_TYPE == 'humans' else df_test
MODEL = ulmfit_fwd

print(MODEL_NAME, DS.shape, DS_TYPE, MODEL)

bar = progress_bar(DS.preprocessed_full_text_first_instance_court_ruling.values)
output_preds = list()
for ruling in bar:
    output_preds.append(MODEL.predict(ruling))


ULMFiT-fwd (690, 38) humans RNNLearner(data=TextClasDataBunch;

Train: LabelList (0 items)
x: TextList

y: CategoryList

Path: /media/nas-elias/tesla4/jefs5/paper/code;

Valid: LabelList (0 items)
x: TextList

y: CategoryList

Path: /media/nas-elias/tesla4/jefs5/paper/code;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(60000, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(60000, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinea

In [8]:
df_preds = pd.Series([int(i[0]) for i in output_preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].replace({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,humans
1,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,humans
2,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,humans
3,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,humans
4,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,humans
...,...,...,...,...
685,NÃO PROVIMENTO,PROVIMENTO,ULMFiT-fwd,humans
686,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,humans
687,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,humans
688,PROVIMENTO,PROVIMENTO,ULMFiT-fwd,humans


In [9]:
MODEL_NAME = 'ULMFiT-fwd'

DS_TYPE = 'test'
DS = df_humans if DS_TYPE == 'humans' else df_test
MODEL = ulmfit_fwd

print(MODEL_NAME, DS.shape, DS_TYPE, MODEL)

bar = progress_bar(DS.preprocessed_full_text_first_instance_court_ruling.values)
output_preds = list()
for ruling in bar:
    output_preds.append(MODEL.predict(ruling))


ULMFiT-fwd (76299, 36) test RNNLearner(data=TextClasDataBunch;

Train: LabelList (0 items)
x: TextList

y: CategoryList

Path: /media/nas-elias/tesla4/jefs5/paper/code;

Valid: LabelList (0 items)
x: TextList

y: CategoryList

Path: /media/nas-elias/tesla4/jefs5/paper/code;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(60000, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(60000, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinea

In [10]:

df_preds = pd.Series([int(i[0]) for i in output_preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].map({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,test
1,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,test
2,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,test
3,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,test
4,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,test
...,...,...,...,...
76294,NÃO PROVIMENTO,PROVIMENTO,ULMFiT-fwd,test
76295,NÃO PROVIMENTO,PROVIMENTO,ULMFiT-fwd,test
76296,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-fwd,test
76297,NÃO PROVIMENTO,PROVIMENTO,ULMFiT-fwd,test


# ULMFiT backward

In [5]:
defaults.device = torch.device('cuda:1')

In [6]:
ulmfit_bwd = load_learner(os.getcwd(), 'saved_models_ulmfit/ulmfit_backward_exported', backwards=True)

In [7]:
MODEL_NAME = 'ULMFiT-bwd'

DS_TYPE = 'humans'
DS = df_humans if DS_TYPE == 'humans' else df_test
MODEL = ulmfit_bwd

print(MODEL_NAME, DS.shape, DS_TYPE, MODEL)

bar = progress_bar(DS.preprocessed_full_text_first_instance_court_ruling.values)
output_preds = list()
for ruling in bar:
    output_preds.append(MODEL.predict(ruling))


ULMFiT-bwd (690, 38) humans RNNLearner(data=TextClasDataBunch;

Train: LabelList (0 items)
x: TextList

y: CategoryList

Path: /media/nas-elias/tesla4/jefs5/paper/code;

Valid: LabelList (0 items)
x: TextList

y: CategoryList

Path: /media/nas-elias/tesla4/jefs5/paper/code;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(60000, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(60000, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinea

In [8]:
df_preds = pd.Series([int(i[0]) for i in output_preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].replace({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,humans
1,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,humans
2,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,humans
3,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,humans
4,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,humans
...,...,...,...,...
685,PROVIMENTO,PROVIMENTO,ULMFiT-bwd,humans
686,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,humans
687,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,humans
688,PROVIMENTO,PROVIMENTO,ULMFiT-bwd,humans


In [9]:
MODEL_NAME = 'ULMFiT-bwd'

DS_TYPE = 'test'
DS = df_humans if DS_TYPE == 'humans' else df_test
MODEL = ulmfit_bwd

print(MODEL_NAME, DS.shape, DS_TYPE, MODEL)

bar = progress_bar(DS.preprocessed_full_text_first_instance_court_ruling.values)
output_preds = list()
for ruling in bar:
    output_preds.append(MODEL.predict(ruling))


ULMFiT-bwd (76299, 36) test RNNLearner(data=TextClasDataBunch;

Train: LabelList (0 items)
x: TextList

y: CategoryList

Path: /media/nas-elias/tesla4/jefs5/paper/code;

Valid: LabelList (0 items)
x: TextList

y: CategoryList

Path: /media/nas-elias/tesla4/jefs5/paper/code;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(60000, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(60000, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinea

In [10]:

df_preds = pd.Series([int(i[0]) for i in output_preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].map({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,test
1,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,test
2,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,test
3,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,test
4,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,test
...,...,...,...,...
76294,PROVIMENTO,PROVIMENTO,ULMFiT-bwd,test
76295,PROVIMENTO,PROVIMENTO,ULMFiT-bwd,test
76296,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bwd,test
76297,PROVIMENTO,PROVIMENTO,ULMFiT-bwd,test


# ULMFiT bidir ( (forward + backward) / 2 )

In [11]:
MODEL_NAME = 'ULMFiT-bidir'

DS_TYPE = 'humans'
DS = df_humans if DS_TYPE == 'humans' else df_test

preds_fwd = joblib.load(f'predictions/raw-ULMFiT-fwd-{DS_TYPE}.joblib')
preds_bwd = joblib.load(f'predictions/raw-ULMFiT-bwd-{DS_TYPE}.joblib')

In [27]:
probs_fwd = torch.stack([i[2] for i in preds_fwd])
probs_bwd = torch.stack([i[2] for i in preds_bwd])
probs_bidir = (probs_bwd + probs_fwd) / 2
preds = np.argmax(probs_bidir, axis = 1)


In [28]:

df_preds = pd.Series([int(i) for i in preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].map({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,humans
1,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,humans
2,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,humans
3,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,humans
4,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,humans
...,...,...,...,...
685,NÃO PROVIMENTO,PROVIMENTO,ULMFiT-bidir,humans
686,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,humans
687,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,humans
688,PROVIMENTO,PROVIMENTO,ULMFiT-bidir,humans


# ULMFiT bidir ( (forward + backward) / 2 )

In [29]:
MODEL_NAME = 'ULMFiT-bidir'

DS_TYPE = 'test'
DS = df_humans if DS_TYPE == 'humans' else df_test

preds_fwd = joblib.load(f'predictions/raw-ULMFiT-fwd-{DS_TYPE}.joblib')
preds_bwd = joblib.load(f'predictions/raw-ULMFiT-bwd-{DS_TYPE}.joblib')

In [30]:
probs_fwd = torch.stack([i[2] for i in preds_fwd])
probs_bwd = torch.stack([i[2] for i in preds_bwd])
probs_bidir = (probs_bwd + probs_fwd) / 2
preds = np.argmax(probs_bidir, axis = 1)


In [31]:

df_preds = pd.Series([int(i) for i in preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].map({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,test
1,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,test
2,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,test
3,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,test
4,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,test
...,...,...,...,...
76294,PROVIMENTO,PROVIMENTO,ULMFiT-bidir,test
76295,PROVIMENTO,PROVIMENTO,ULMFiT-bidir,test
76296,NÃO PROVIMENTO,NÃO PROVIMENTO,ULMFiT-bidir,test
76297,PROVIMENTO,PROVIMENTO,ULMFiT-bidir,test


# BERT + LSTM

In [7]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

class MCC(Metric):
 
    def __init__(
        self,
        compute_on_step: bool = True,
        dist_sync_on_step: bool = False,
        process_group: Optional[Any] = None,
    ):
        super().__init__(
            compute_on_step=compute_on_step,
            dist_sync_on_step=dist_sync_on_step,
            process_group=process_group,
        )
        self.add_state("preds", default=[], dist_reduce_fx=None)
        self.add_state("target", default=[], dist_reduce_fx=None)


    def update(self, preds: torch.Tensor, target: torch.Tensor):
        self.preds.append(preds.flatten().long())
        self.target.append(target.flatten().long())

    def compute(self) -> Union[Tuple[torch.Tensor, torch.Tensor, torch.Tensor],
                               Tuple[List[torch.Tensor], List[torch.Tensor], List[torch.Tensor]]]:

        preds = torch.cat(self.preds, dim=0).cpu().numpy()
        target = torch.cat(self.target, dim=0).cpu().numpy()
        return matthews_corrcoef(preds, target), preds, target


class ModelBERTLSTM(nn.Module):

    def __init__(self, model_name: str, bertconfig: BertConfig, n_layers_lstm:int, bidir_lstm:bool, drop_mult: float, use_special_classifier:str, output_lstm_size:int, distil_init: bool):

        super().__init__()

        self.bert = transformers.BertModel.from_pretrained(model_name)#.bert
        self.bert.config = bertconfig
        self.dropout_mult = drop_mult
        self.dropout = nn.Dropout(p=self.dropout_mult)

        self.lstm = nn.LSTM(self.bert.config.hidden_size * 4,
                            output_lstm_size,
                            num_layers=n_layers_lstm,
                            dropout=self.dropout_mult if n_layers_lstm > 1 else 0,
                            bidirectional=bidir_lstm)

        sizes_classifier = [output_lstm_size * (2 if bidir_lstm else 1), int(self.bert.config.hidden_size//2), 1]

        if use_special_classifier == 'ln': 
            self.classifier = nn.Sequential(
                self.dropout,
                nn.Linear(sizes_classifier[0], sizes_classifier[1]),
                nn.LayerNorm(sizes_classifier[1]),
                Mish(),
                self.dropout,
                nn.Linear(sizes_classifier[1], sizes_classifier[2]),
                nn.LayerNorm(sizes_classifier[2])
            )
        elif use_special_classifier == 'bn': 
            self.classifier = nn.Sequential(
                self.dropout,
                nn.Linear(sizes_classifier[0], sizes_classifier[1]),
                nn.BatchNorm1d(sizes_classifier[1]),
                Mish(),
                self.dropout,
                nn.Linear(sizes_classifier[1], sizes_classifier[2]),
                nn.BatchNorm1d(sizes_classifier[2])
            )
        elif use_special_classifier == 'none': 
            self.classifier = nn.Sequential(
                self.dropout,
                nn.Linear(sizes_classifier[0], sizes_classifier[1]),
                Mish(),
                self.dropout,
                nn.Linear(sizes_classifier[1], sizes_classifier[2])
            )

        # DistilBERT’s custom initialization
        # https://zablo.net/blog/post/custom-classifier-on-bert-model-guide-polemo2-sentiment-analysis/

        if distil_init:
            for layer in self.classifier:
                if isinstance(layer, nn.Linear):
                    layer.weight.data.normal_(mean=0.0, std=0.02)
                    if layer.bias is not None:
                        layer.bias.data.zero_()

    #input_ids, token_type_ids, attention_masks
    def forward(self, document_batch: torch.Tensor, document_sequence_lengths: list, device='cuda'):

        # contains all BERT sequences
        # bert should output a (batch_size, num_sequences, bert_hidden_size)
        bert_output = torch.zeros(size=(document_batch.shape[0],
                                           min(document_batch.shape[1], self.bert.config.batch_size),
                                           self.bert.config.hidden_size*4), dtype=torch.float, device=device)
       # pdb.set_trace() # DEBUG
        # only pass through bert_batch_size numbers of inputs into bert. #
        # this means that we are possibly cutting off the last part of documents.

        for doc_id in range(document_batch.shape[0]):
            out_bert = self.bert(document_batch[doc_id][:self.bert.config.batch_size, 0],
                                       token_type_ids=document_batch[doc_id][:self.bert.config.batch_size, 1],
                                       attention_mask=document_batch[doc_id][:self.bert.config.batch_size, 2], output_hidden_states=True)#[0][:, 0, :]


            hidden_states = out_bert[2]
            h12 = hidden_states[-1][:,0]
            h11 = hidden_states[-2][:,0]
            h10 = hidden_states[-3][:,0]
            h09 = hidden_states[-4][:,0]
            concat_hidden = torch.cat([h09, h10, h11, h12], axis=-1)
            
            bert_output[doc_id][:self.bert.config.batch_size] = concat_hidden

        lista = []
        for idx, i in enumerate(document_sequence_lengths+1):
            lista.append(bert_output[idx, :int(i)])
        packed = pack_sequence(lista, enforce_sorted=False)        
        self.lstm.flatten_parameters()
        output, (_, _) = self.lstm(packed)
        output, _ = pad_packed_sequence(output) # TODO ESTUDAR MELHOR ESSA SAIDA
        indexes =  [len(i) -1 for i in lista]  # x is the input, which is a list contains sentence embeddings in a batch

        last_layer = output[indexes, range(output.shape[1]), :]
        out = self.classifier(last_layer)
        return out

    def freeze_bert_encoder(self):
        print('Freezing all bert encoder')
        for param in self.bert.parameters():
            param.requires_grad = False

    def unfreeze_bert_encoder(self):
        print('Unreezing all bert encoder')
        for param in self.bert.parameters():
            param.requires_grad = True

    def unfreeze_bert_encoder_last_layers(self):
        print('Unfreezing bert encoder last layers')
        for name, param in self.bert.named_parameters():
            if "encoder.layer.11" in name or "pooler" in name:
                param.requires_grad = True

    def unfreeze_bert_encoder_pooler_layer(self):
        print('Unfreezing bert encoder last pooler layer')
        for name, param in self.bert.named_parameters():
            if "pooler" in name:
                print(name)
                param.requires_grad = True



class EncodeCollateFn:


    def __init__(self, tokenizer: AutoTokenizer, bert_batch_size, max_input_length=512):

        self.tokenizer = tokenizer
        self.max_tokens = max_input_length
        self.bert_batch_size = bert_batch_size

    def get_sliding_window_tokens(self, text: str):
        cut = self.bert_batch_size
        raw_tokens = self.tokenizer.tokenize(text)
        tokens = {}
        input_type_ids = {}
        attention_masks = {}

        if len(raw_tokens) > (self.max_tokens-2)*cut:
            max_inicio = math.floor(cut/2)
            max_fim = math.ceil(cut/2)
            all_tokens = raw_tokens[:(self.max_tokens-2)*max_inicio] + raw_tokens[-(self.max_tokens-2)*max_fim:]
            was_cut = True
        else:
            all_tokens = raw_tokens
            was_cut = False

        def chunks(lista, n):
            n = max(1, n)
            return [lista[i:i+n] for i in range(0, len(lista), n)]

        def fill(lista, desired_length, filler=0):
            if len(lista) >= desired_length:
                return lista

            current_size = len(lista)
            n = desired_length - current_size
            listofzeros = [filler] * n
            lista += listofzeros
            return lista
            
        tokens_split = chunks(all_tokens, self.max_tokens-2)
        tokens_split = [["CLS"] + chunk + ["[SEP"] for chunk in tokens_split]

        for n, split in enumerate(tokens_split):
            tokens[n] = np.array(fill(self.tokenizer.convert_tokens_to_ids(split), self.max_tokens, self.tokenizer.pad_token_id))
            input_type_ids[n] = np.array(fill([0] * len(split), self.max_tokens, 0))
            attention_masks[n] = np.array(fill([1] * len(split), self.max_tokens, 0))

        retorno = {
            'original_len': len(raw_tokens),
            'n_chunks': len(tokens.keys()),
            'tokenized_chunks': {
                'input_ids': tokens,
                'input_type_ids': input_type_ids,
                'attention_masks': attention_masks
            },
            'was_cut': was_cut
        }
        return retorno    


    def __call__(self, batch):

        documents = [x[0] for x in batch]

        assert type(documents) == list, 'Needs to be a list of strings'

        split_documents = list()

        for document in documents:
            split_documents.append(self.get_sliding_window_tokens(text=document))

        max_sequences_per_document = math.ceil(max([i['n_chunks'] for i in split_documents]))

        assert max_sequences_per_document <= 15, f"Your document is way too large"

        output = torch.zeros(size=(len(documents), max_sequences_per_document, 3, 512), dtype=torch.long)
        document_seq_lengths = []  # number of sequence generated per document

        for doc_index, split_document in enumerate(split_documents):
            for seq_index in range(split_document['n_chunks']):
                    
                input_ids = split_document['tokenized_chunks']['input_ids'][seq_index]#.squeeze()
                attention_masks = split_document['tokenized_chunks']['attention_masks'][seq_index]#.squeeze()
                input_type_ids = split_document['tokenized_chunks']['attention_masks'][seq_index]#.squeeze()
    

                assert len(input_ids) == self.max_tokens and len(attention_masks) == self.max_tokens and len(input_type_ids) == self.max_tokens, f"{len(input_ids)} - {len(attention_masks)} - {len(input_type_ids)}"

                # we are ready to rumble
                output[doc_index][seq_index] = torch.cat((torch.LongTensor(input_ids).unsqueeze(0),
                                                          torch.LongTensor(input_type_ids).unsqueeze(0),
                                                          torch.LongTensor(attention_masks).unsqueeze(0)),
                                                         dim=0)
                max_seq_index = seq_index
            #document_seq_lengths.append(max_seq_index+1)
            document_seq_lengths.append(split_document['n_chunks'])

        labels = torch.tensor([x[1] for x in batch], dtype=torch.int8)
        return (output, torch.LongTensor(document_seq_lengths)), labels
    

# from https://github.com/digantamisra98/Mish/blob/b5f006660ac0b4c46e2c6958ad0301d7f9c59651/Mish/Torch/mish.py
@torch.jit.script
def mish(input):
    return input * torch.tanh(F.softplus(input))

class Mish(nn.Module):
    def forward(self, input):
        return mish(input)


class JEFDataset(Dataset):
    def __init__(self, path, dep_var, text_col, lowercase):
        super().__init__()
        self.dep_var = dep_var
        self.text_col = text_col
        all_columns = [self.dep_var, self.text_col] + ['date_appeal_panel_ruling']

        data = pd.read_parquet(path, columns=all_columns)    

        if len(data) > 600_000:
            print(f'Previous size of training data: {len(data)}. Selecting only last 5 years of the training dataset')
            data.date_appeal_panel_ruling = pd.to_datetime(data.date_appeal_panel_ruling, infer_datetime_format=True, yearfirst=True, dayfirst=False)
            thresh = data.date_appeal_panel_ruling.max() - timedelta(days=365*5)
            data = data[data.date_appeal_panel_ruling >= thresh].copy()
            print(f'New size of training data: {len(data)}')

        data.drop('date_appeal_panel_ruling', axis=1, inplace=True)
        data[self.dep_var] = data[self.dep_var].replace('PROVIMENTO PARCIAL', 'PROVIMENTO')
        data = data[data[self.dep_var].isin(['PROVIMENTO', 'NÃO PROVIMENTO'])]
        data[self.dep_var] = data[self.dep_var].map({'NÃO PROVIMENTO': 0, 'PROVIMENTO': 1})

        if lowercase:
            data[self.text_col] = data[self.text_col].str.lower()

        print(f'Size before: {len(data)} - {path.split("/")[-1]}')        
        data.dropna(inplace=True)
        print(f'Size after: {len(data)} - {path.split("/")[-1]}')
        data.reset_index(drop=True, inplace=True)
        self.data = data.copy()

    def __getitem__(self, idx):
        return self.data.loc[idx, self.text_col], self.data.loc[idx, self.dep_var]

    def __len__(self):
        return len(self.data)

class TrainingModule(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        self.num_labels = len(self.hparams['labels'])
        self.tokenizer = AutoTokenizer.from_pretrained(self.hparams['bert_model_path'])
        config = AutoConfig.from_pretrained(self.hparams['bert_model_path'])
        config.__setattr__('num_labels', len(self.hparams['labels']))
        config.__setattr__('batch_size', self.hparams['bert_batch_size'])

        self.accuracy = torchmetrics.Accuracy()
        self.mcc = MCC()
        self.valor_mcc = {'val_mcc': -1, 'test_mcc': -1}
        self.best_mcc = -1.0
        self.precision_metric = torchmetrics.Precision(num_classes=self.num_labels)
        self.recall_metric = torchmetrics.Recall(num_classes=self.num_labels)
        #self.confmat = torchmetrics.ConfusionMatrix(num_classes=self.num_labels)
        self.f1_score = torchmetrics.F1(num_classes=self.num_labels)

        self.model = ModelBERTLSTM(self.hparams['bert_model_path'],
                                      bertconfig=config,
                                      n_layers_lstm=self.hparams.n_layers_lstm,
                                      bidir_lstm=self.hparams.bidir_lstm,
                                      drop_mult=self.hparams.drop_mult,
                                      use_special_classifier=self.hparams.use_special_classifier,
                                      output_lstm_size=self.hparams.output_lstm_size,
                                      distil_init=self.hparams.distil_init)


        if self.hparams.bert_unfreeze_mode == 'encoder_last':
            self.model.freeze_bert_encoder()
            self.model.unfreeze_bert_encoder_last_layers()
        elif self.hparams.bert_unfreeze_mode == 'pooler_last':
            self.model.freeze_bert_encoder()
            self.model.unfreeze_bert_encoder_pooler_layer()
        elif self.hparams.bert_unfreeze_mode == 'all':
            self.model.unfreeze_bert_encoder()
        elif self.hparams.bert_unfreeze_mode == 'none':
            self.model.freeze_bert_encoder()

        if self.hparams.weighted_loss:
            weights = torch.FloatTensor(self.hparams.train_weights)
            print(f'Using weighted loss: {weights}')
        else:
            weights = None
        # weight = torch.FloatTensor(self.hparams.train_weights).to(self.hparams.device) if self.hparams.train_weights is not None else None)
        self.loss = nn.BCEWithLogitsLoss(pos_weight=weights)
        self.lr = self.hparams.lr
        self.save_hyperparameters()

    def step(self, batch, step_name='train'):
        thresh = self.hparams.thresh_step
        input_ids, attention_masks, y = batch
        logits = self.forward(input_ids, attention_masks).squeeze()
        y = y.type_as(logits)
        loss = self.loss(logits, y)

        if step_name == 'train':
            self.log('train_loss', loss, on_step=True, on_epoch=True,
                     logger=True, prog_bar=True, sync_dist=True, sync_dist_op='mean')
            result = {'train_loss': loss}
            return loss

        else:
            self.log(f'{step_name}_loss', loss, on_step=False, on_epoch=True, logger=True,
                     prog_bar=True, reduce_fx=torch.mean, sync_dist=True, sync_dist_op='mean')

            y_pred = torch.sigmoid(logits)
            y_pred = torch.where(y_pred > thresh, 1.0, 0.0).long()
            y =  y.long()
            #y_pred = torch.argmax(logits, dim=1)
            self.accuracy(y_pred, y)
            #self.mymetric(y_pred, y)
            self.precision_metric(y_pred, y)
            self.recall_metric(y_pred, y)
            self.f1_score(y_pred, y)
            #self.confmat(y_pred, y)
            self.mcc(y_pred, y)

    
        result = {f'{step_name}_loss': loss}
        return result
        
    def calculate_metrics(self, outputs, step_name='val'):
        
        mcc, preds, target = self.mcc.compute()
        tn = ((target == preds) & (target == 0)).sum()
        tp = ((target == preds) & (target == 1)).sum()
        fn = ((target != preds) & (target == 1)).sum()
        fp = ((target != preds) & (target == 0)).sum()

        
        outs = {}
        outs[f'{step_name}_acc'] = self.accuracy.compute()
        outs[f'{step_name}_loss'] = torch.mean(torch.tensor([i[f'{step_name}_loss'] for i in outputs]))
        outs[f'{step_name}_tn'] = tn
        outs[f'{step_name}_fp'] = fp
        outs[f'{step_name}_fn'] = fn
        outs[f'{step_name}_tp'] = tp
        outs[f'{step_name}_f1_score'] = self.f1_score.compute()
        outs[f'{step_name}_precision'] = self.precision_metric.compute()
        outs[f'{step_name}_recall'] = self.recall_metric.compute()
        outs[f'{step_name}_mcc'] = mcc
        #outs[f'{step_name}_mccold'] = mcc2
        #confmat = self.confmat.compute().long().detach().cpu().numpy()

        self.recall_metric.reset()
        self.precision_metric.reset()
        self.f1_score.reset()
        self.accuracy.reset()
        self.mcc.reset()
        #self.confmat.reset()

        if float(mcc) > self.best_mcc:
            self.best_mcc = float(mcc)
            self.log('best_mcc', mcc)
        if self.valor_mcc[f'{step_name}_mcc'] < float(outs[f'{step_name}_mcc']):
            self.valor_mcc[f'{step_name}_mcc'] = float(outs[f'{step_name}_mcc'])

        print(matthews_corrcoef(preds, target), mcc, len(target), len(preds))
        if self.global_rank == 0:
            print(matthews_corrcoef(preds, target), mcc, len(target), len(preds))
            if self.valor_mcc[f'{step_name}_mcc'] < float(outs[f'{step_name}_mcc']):
                self.valor_mcc[f'{step_name}_mcc'] = float(outs[f'{step_name}_mcc'])
                #self.logger.experiment.log({f"best_mcc-confusion_matrix" : wandb.plot.confusion_matrix(preds=preds, y_true=target, class_names=[i[:3].upper() for i in self.hparams.labels])}, commit=False)
            #print("\n\nCONFUSION MATRIX:\n", confmat, "\n")
            print(f"{step_name}_acc: {float(outs[f'{step_name}_acc']):.5f}")
            print(f"{step_name}_mcc: {float(outs[f'{step_name}_mcc']):.5f}")
            print(f"Number of cases: {int(tn+fp+fn+tp)}")
            print('\n')


        for k, v in outs.items():
            self.log(k, v)

    def forward(self, X_text, *args):
        return self.model(X_text[0], X_text[1], *args)
    
    def training_step(self, batch, batch_idx):
        return self.step(batch, "train")

    def validation_step(self, batch, batch_idx):
        return self.step(batch, "val")

    def test_step(self, batch, batch_idx):
        return self.step(batch, "test")

    def validation_epoch_end(self, outputs: List[dict]):
        return self.calculate_metrics(outputs, step_name='val')

    def test_epoch_end(self, outputs: List[dict]):
        return self.calculate_metrics(outputs, step_name='test')

    def train_dataloader(self):
        return self.create_data_loader(self.hparams.train_path, shuffle=True)

    def val_dataloader(self):
        return self.create_data_loader(self.hparams.val_path)

    def test_dataloader(self):
        return self.create_data_loader(self.hparams.test_path)

    def create_data_loader(self, ds_path: str, shuffle=False):
        #print(self.hparams.cat_names)
        return DataLoader(
            JEFDataset(ds_path, self.hparams.dep_var, self.hparams.text_col, self.hparams.lowercase), 
            batch_size=self.hparams.batch_size,
            shuffle=shuffle,
            pin_memory=True,
            drop_last=True,
            num_workers=int(cpu_count()),
            collate_fn=EncodeCollateFn(self.tokenizer, self.hparams.bert_batch_size)
        )

    def setup(self, stage):
        if stage == 'fit':
            total_devices = max(1, self.hparams.n_gpus) * 1
            train_batches = len(self.train_dataloader()) // total_devices
            self.train_steps = math.ceil((self.hparams.epochs * train_batches) / self.hparams.accumulate_grad_batches)
            self.train_steps = int(math.ceil(self.train_steps * 1.01))#1.04)

    def configure_optimizers(self):
        train_steps = self.train_steps
        optimizer = torch.optim.AdamW([p for p in self.model.parameters() if p.requires_grad], lr=self.lr, weight_decay=0.1)
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                           max_lr=self.lr,
                                                           total_steps=train_steps,
                                                           three_phase=True,
                                                           epochs=self.hparams.epochs)

        return [optimizer], [{"scheduler": lr_scheduler, "interval": "step"}]


In [8]:
learn = TrainingModule.load_from_checkpoint('.//bert_lstm-best.ckpt')

/home/jacob/miniconda3/envs/paper_brcad5/lib/python3.9/site-packages/deprecate/deprecation.py:115: FutureWarning: The `F1` was deprecated since v0.7 in favor of `torchmetrics.classification.f_beta.F1Score`. It will be removed in v0.8.
  stream(template_mgs % msg_args)
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model tha

Freezing all bert encoder
Unfreezing bert encoder last layers
Using weighted loss: 3.8351831436157227


In [9]:
learn.freeze()
learn.eval();

In [10]:
learn.to('cuda:0'); 

In [11]:
learn.device

device(type='cuda', index=0)

In [12]:
learn.hparams

"accumulate_grad_batches": 3
"batch_size":              8
"bert_batch_size":         15
"bert_model_path":         neuralmind/bert-base-portuguese-cased
"bert_unfreeze_mode":      encoder_last
"bidir_lstm":              False
"dep_var":                 label
"deterministic":           True
"device":                  cuda
"distil_init":             False
"drop_mult":               0.1
"epochs":                  8
"gradient_clip_val":       0.0
"hparams":                 Namespace(train_path='../data/train_en.parquet', val_path='../data/valid_en.parquet', test_path='../data/test_en.parquet', batch_size=8, epochs=8, drop_mult=0.1, n_layers_lstm=1, bidir_lstm=False, use_special_classifier='none', output_lstm_size=1536, lowercase=False, lr=0.0003, lr_divider=15, thresh_step=0.49, accumulate_grad_batches=3, gradient_clip_val=0.0, bert_batch_size=15, distil_init=False, stochastic_weight_avg=True, use_metadata=True, dep_var='label', text_col='preprocessed_full_text_first_instance_court_ruling'

In [28]:
def predict(model, text):
    output, _ = EncodeCollateFn(model.tokenizer, model.hparams.bert_batch_size)([[text, 0]])
    if model.device != 'cpu':
        output = (output[0].to(model.device), output[1].to(model.device))
    logits = model.forward(output)
    prob  = torch.sigmoid(logits)
    y_pred = torch.where(prob > model.hparams.thresh_step, 1.0, 0.0).long()
    return logits.cpu().numpy(), prob.cpu().numpy(), int(y_pred)


In [29]:
MODEL_NAME = 'BERT_LSTM'
DS_TYPE = 'humans'
DS = df_humans if DS_TYPE == 'humans' else df_test

output_preds = []
bar = progress_bar(DS.preprocessed_full_text_first_instance_court_ruling.values)

for ruling in bar:
    output_preds.append(predict(learn, ruling))
    

In [32]:
output_preds[1][2]

0

In [33]:
df_preds = pd.Series([i[2] for i in output_preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].replace({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,humans
1,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,humans
2,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,humans
3,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,humans
4,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,humans
...,...,...,...,...
685,NÃO PROVIMENTO,PROVIMENTO,BERT_LSTM,humans
686,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,humans
687,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,humans
688,PROVIMENTO,PROVIMENTO,BERT_LSTM,humans


In [34]:
MODEL_NAME = 'BERT_LSTM'
DS_TYPE = 'test'
DS = df_humans if DS_TYPE == 'humans' else df_test

output_preds = []
bar = progress_bar(DS.preprocessed_full_text_first_instance_court_ruling.values)

for ruling in bar:
    output_preds.append(predict(learn, ruling))
    

In [35]:
df_preds = pd.Series([i[2] for i in output_preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].replace({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,test
1,PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,test
2,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,test
3,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,test
4,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,test
...,...,...,...,...
76294,PROVIMENTO,PROVIMENTO,BERT_LSTM,test
76295,NÃO PROVIMENTO,PROVIMENTO,BERT_LSTM,test
76296,NÃO PROVIMENTO,NÃO PROVIMENTO,BERT_LSTM,test
76297,PROVIMENTO,PROVIMENTO,BERT_LSTM,test


# BigBird

In [6]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

seed_everything(314)

class MCC(Metric):
 
    def __init__(
        self,
        compute_on_step: bool = True,
        dist_sync_on_step: bool = False,
        process_group: Optional[Any] = None,
    ):
        super().__init__(
            compute_on_step=compute_on_step,
            dist_sync_on_step=dist_sync_on_step,
            process_group=process_group,
        )
        self.add_state("preds", default=[], dist_reduce_fx=None)
        self.add_state("target", default=[], dist_reduce_fx=None)


    def update(self, preds: torch.Tensor, target: torch.Tensor):
        self.preds.append(preds.flatten().long())
        self.target.append(target.flatten().long())

    def compute(self) -> Union[Tuple[torch.Tensor, torch.Tensor, torch.Tensor],
                               Tuple[List[torch.Tensor], List[torch.Tensor], List[torch.Tensor]]]:

        preds = torch.cat(self.preds, dim=0).cpu().numpy()
        target = torch.cat(self.target, dim=0).cpu().numpy()
        return matthews_corrcoef(preds, target), preds, target


class ModelBigBird(nn.Module):

    def __init__(self, model_name: str, bertconfig: BertConfig, drop_mult: float, use_special_classifier:str):

        super().__init__()
        self.bert = transformers.AutoModel.from_pretrained(model_name, add_pooling_layer=False)
        self.bert.config = bertconfig
        self.dropout_mult = drop_mult
        self.dropout = nn.Dropout(self.dropout_mult)

        sizes_classifier = [self.bert.config.hidden_size*4, self.bert.config.hidden_size, int(self.bert.config.hidden_size//2), 1]

        if use_special_classifier == 'ln': 
            self.classifier = nn.Sequential(
                self.dropout,
                nn.Linear(sizes_classifier[0], sizes_classifier[1]),
                nn.LayerNorm(sizes_classifier[1]),
                Mish(),
                self.dropout,
                nn.Linear(sizes_classifier[1], sizes_classifier[2]),
                nn.LayerNorm(sizes_classifier[2]),
                Mish(),
                self.dropout,
                nn.Linear(sizes_classifier[2], sizes_classifier[3]),
                nn.LayerNorm(sizes_classifier[3]),
            )

        elif use_special_classifier == 'bn': 
            self.classifier = nn.Sequential(
                self.dropout,
                nn.Linear(sizes_classifier[0], sizes_classifier[1]),
                nn.BatchNorm1d(sizes_classifier[1]),
                Mish(),
                self.dropout,
                nn.Linear(sizes_classifier[1], sizes_classifier[2]),
                nn.BatchNorm1d(sizes_classifier[2]),
                Mish(),
                self.dropout,
                nn.Linear(sizes_classifier[2], sizes_classifier[3]),
                nn.BatchNorm1d(sizes_classifier[3]),
            )

        elif use_special_classifier == 'none': 
            self.classifier = nn.Sequential(
                self.dropout,
                nn.Linear(sizes_classifier[0], sizes_classifier[1]),
                Mish(),
                self.dropout,
                nn.Linear(sizes_classifier[1], sizes_classifier[2]),
                Mish(),
                self.dropout,
                nn.Linear(sizes_classifier[2], sizes_classifier[3])
            )

    def forward(self, input_ids: torch.Tensor, attention_masks: torch.Tensor, device='cuda'):
        out_bert = self.bert(input_ids, attention_masks, output_hidden_states=True)
        hidden_states = out_bert[1]
        h12 = hidden_states[-1][:,0]
        h11 = hidden_states[-2][:,0]
        h10 = hidden_states[-3][:,0]
        h09 = hidden_states[-4][:,0]
        concat_hidden = torch.cat([h09, h10, h11, h12], axis=-1)
        out = self.classifier(concat_hidden)
        return out

    def freeze_bert_encoder(self):
        print('Freezing all bert encoder')
        for param in self.bert.parameters():
            param.requires_grad = False

    def unfreeze_bert_encoder(self):
        print('Unreezing all bert encoder')
        for param in self.bert.parameters():
            param.requires_grad = True

    def unfreeze_bert_encoder_last_layers(self):
        print('Unfreezing bert encoder last layers')
        for name, param in self.bert.named_parameters():
            if "encoder.layer.11" in name or "pooler" in name:
                param.requires_grad = True

    def unfreeze_bert_encoder_pooler_layer(self):
        print('Unfreezing bert encoder last pooler layer')
        for name, param in self.bert.named_parameters():
            if "pooler" in name:
                print(name)
                param.requires_grad = True



class EncodeCollateFn:

    def slice_text(self, text):
        split = text.split()
        size = len(split)
        if size > self.max_tokens:
            new_text = split[:self.max_tokens//2] + split[-self.max_tokens//2:]
            text = ' '.join(new_text)
        return text

    def __init__(self, tokenizer: AutoTokenizer, max_input_length=7680):

        self.tokenizer = tokenizer
        self.max_tokens = max_input_length

    def __call__(self, batch):

        documents = [self.slice_text(x[0]) for x in batch]
        labels = torch.tensor([x[1] for x in batch], dtype=torch.int8)

        assert type(documents) == list, 'Needs to be a list of strings'
        tokenized = self.tokenizer(documents, return_tensors='pt', padding=True, truncation=True, max_length=self.max_tokens)

        return tokenized['input_ids'], tokenized['attention_mask'], labels


# from https://github.com/digantamisra98/Mish/blob/b5f006660ac0b4c46e2c6958ad0301d7f9c59651/Mish/Torch/mish.py
@torch.jit.script
def mish(input):
    return input * torch.tanh(F.softplus(input))

class Mish(nn.Module):
    def forward(self, input):
        return mish(input)


class JEFDataset(Dataset):
    def __init__(self, path, dep_var, text_col, lowercase):
        super().__init__()
        self.dep_var = dep_var
        self.text_col = text_col
        all_columns = [self.dep_var, self.text_col] + ['date_appeal_panel_ruling']

        data = pd.read_parquet(path, columns=all_columns)    

        if len(data) > 600_000:
            print(f'Previous size of training data: {len(data)}. Selecting only last 5 years of the training dataset')
            data.date_appeal_panel_ruling = pd.to_datetime(data.date_appeal_panel_ruling, infer_datetime_format=True, yearfirst=True, dayfirst=False)
            thresh = data.date_appeal_panel_ruling.max() - timedelta(days=365*5)
            data = data[data.date_appeal_panel_ruling >= thresh].copy()
            print(f'New size of training data: {len(data)}')

        data.drop('date_appeal_panel_ruling', axis=1, inplace=True)
        data[self.dep_var] = data[self.dep_var].replace('PROVIMENTO PARCIAL', 'PROVIMENTO')
        data = data[data[self.dep_var].isin(['PROVIMENTO', 'NÃO PROVIMENTO'])]
        data[self.dep_var] = data[self.dep_var].map({'NÃO PROVIMENTO': 0, 'PROVIMENTO': 1})

        if lowercase:
            data[self.text_col] = data[self.text_col].str.lower()

        print(f'Size before: {len(data)} - {path.split("/")[-1]}')        
        data.dropna(inplace=True)
        print(f'Size after: {len(data)} - {path.split("/")[-1]}')
        data.reset_index(drop=True, inplace=True)
        self.data = data.copy()

    def __getitem__(self, idx):
        return self.data.loc[idx, self.text_col], self.data.loc[idx, self.dep_var]

    def __len__(self):
        return len(self.data)

class TrainingModule(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        self.num_labels = len(self.hparams['labels'])
        self.tokenizer = BigBirdTokenizer.from_pretrained(self.hparams['bert_model_path'])
        config = AutoConfig.from_pretrained(self.hparams['bert_model_path'])
        config.__setattr__('num_labels', len(self.hparams['labels']))

        self.accuracy = torchmetrics.Accuracy()
        self.mcc = MCC()
        self.valor_mcc = {'val_mcc': -1, 'test_mcc': -1}
        self.best_mcc = -1.0
        self.precision_metric = torchmetrics.Precision(num_classes=self.num_labels)
        self.recall_metric = torchmetrics.Recall(num_classes=self.num_labels)
        #self.confmat = torchmetrics.ConfusionMatrix(num_classes=self.num_labels)
        self.f1_score = torchmetrics.F1(num_classes=self.num_labels)

        self.model = ModelBigBird(self.hparams['bert_model_path'],
                                      bertconfig=config,
                                      drop_mult=self.hparams.drop_mult,
                                      use_special_classifier=self.hparams.use_special_classifier)

        if self.hparams.bert_unfreeze_mode == 'encoder_last':
            self.model.freeze_bert_encoder()
            self.model.unfreeze_bert_encoder_last_layers()
        elif self.hparams.bert_unfreeze_mode == 'pooler_last':
            self.model.freeze_bert_encoder()
            self.model.unfreeze_bert_encoder_pooler_layer()
        elif self.hparams.bert_unfreeze_mode == 'all':
            self.model.unfreeze_bert_encoder()
        elif self.hparams.bert_unfreeze_mode == 'none':
            self.model.freeze_bert_encoder()

        if self.hparams.weighted_loss:
            weights = torch.FloatTensor(self.hparams.train_weights)
            print(f'Using weighted loss: {weights}')
        else:
            weights = None
        # weight = torch.FloatTensor(self.hparams.train_weights).to(self.hparams.device) if self.hparams.train_weights is not None else None)
        self.loss = nn.BCEWithLogitsLoss(pos_weight=weights)
        self.lr = self.hparams.lr
        self.save_hyperparameters()

    def step(self, batch, step_name='train'):
        thresh = self.hparams.thresh_step
        input_ids, attention_masks, y = batch
        logits = self.forward(input_ids, attention_masks).squeeze()
        y = y.type_as(logits)
        loss = self.loss(logits, y)

        if step_name == 'train':
            self.log('train_loss', loss, on_step=True, on_epoch=True,
                     logger=True, prog_bar=True, sync_dist=True, sync_dist_op='mean')
            result = {'train_loss': loss}
            return loss

        else:
            self.log(f'{step_name}_loss', loss, on_step=False, on_epoch=True, logger=True,
                     prog_bar=True, reduce_fx=torch.mean, sync_dist=True, sync_dist_op='mean')

            y_pred = torch.sigmoid(logits)
            y_pred = torch.where(y_pred > thresh, 1.0, 0.0).long()
            y =  y.long()
            #y_pred = torch.argmax(logits, dim=1)
            self.accuracy(y_pred, y)
            #self.mymetric(y_pred, y)
            self.precision_metric(y_pred, y)
            self.recall_metric(y_pred, y)
            self.f1_score(y_pred, y)
            #self.confmat(y_pred, y)
            self.mcc(y_pred, y)

    
        result = {f'{step_name}_loss': loss}
        return result
        
    def calculate_metrics(self, outputs, step_name='val'):
        
        mcc, preds, target = self.mcc.compute()
        tn = ((target == preds) & (target == 0)).sum()
        tp = ((target == preds) & (target == 1)).sum()
        fn = ((target != preds) & (target == 1)).sum()
        fp = ((target != preds) & (target == 0)).sum()

        
        outs = {}
        outs[f'{step_name}_acc'] = self.accuracy.compute()
        outs[f'{step_name}_loss'] = torch.mean(torch.tensor([i[f'{step_name}_loss'] for i in outputs]))
        outs[f'{step_name}_tn'] = tn
        outs[f'{step_name}_fp'] = fp
        outs[f'{step_name}_fn'] = fn
        outs[f'{step_name}_tp'] = tp
        outs[f'{step_name}_f1_score'] = self.f1_score.compute()
        outs[f'{step_name}_precision'] = self.precision_metric.compute()
        outs[f'{step_name}_recall'] = self.recall_metric.compute()
        outs[f'{step_name}_mcc'] = mcc
        #outs[f'{step_name}_mccold'] = mcc2
        #confmat = self.confmat.compute().long().detach().cpu().numpy()

        self.recall_metric.reset()
        self.precision_metric.reset()
        self.f1_score.reset()
        self.accuracy.reset()
        self.mcc.reset()
        #self.confmat.reset()

        if float(mcc) > self.best_mcc:
            self.best_mcc = float(mcc)
            self.log('best_mcc', mcc)
        if self.valor_mcc[f'{step_name}_mcc'] < float(outs[f'{step_name}_mcc']):
            self.valor_mcc[f'{step_name}_mcc'] = float(outs[f'{step_name}_mcc'])

        print(matthews_corrcoef(preds, target), mcc, len(target), len(preds))
        if self.global_rank == 0:
            print(matthews_corrcoef(preds, target), mcc, len(target), len(preds))
            if self.valor_mcc[f'{step_name}_mcc'] < float(outs[f'{step_name}_mcc']):
                self.valor_mcc[f'{step_name}_mcc'] = float(outs[f'{step_name}_mcc'])
                #self.logger.experiment.log({f"best_mcc-confusion_matrix" : wandb.plot.confusion_matrix(preds=preds, y_true=target, class_names=[i[:3].upper() for i in self.hparams.labels])}, commit=False)
            #print("\n\nCONFUSION MATRIX:\n", confmat, "\n")
            print(f"{step_name}_acc: {float(outs[f'{step_name}_acc']):.5f}")
            print(f"{step_name}_mcc: {float(outs[f'{step_name}_mcc']):.5f}")
            print(f"Number of cases: {int(tn+fp+fn+tp)}")
            print('\n')


        for k, v in outs.items():
            self.log(k, v)

    def forward(self, input_ids, attention_masks, *args):
        return self.model(input_ids, attention_masks, *args)

    def training_step(self, batch, batch_idx):
        return self.step(batch, "train")

    def validation_step(self, batch, batch_idx):
        return self.step(batch, "val")

    def test_step(self, batch, batch_idx):
        return self.step(batch, "test")

    def validation_epoch_end(self, outputs: List[dict]):
        return self.calculate_metrics(outputs, step_name='val')

    def test_epoch_end(self, outputs: List[dict]):
        return self.calculate_metrics(outputs, step_name='test')

    def train_dataloader(self):
        return self.create_data_loader(self.hparams.train_path, shuffle=True)

    def val_dataloader(self):
        return self.create_data_loader(self.hparams.val_path)

    def test_dataloader(self):
        return self.create_data_loader(self.hparams.test_path)

    def create_data_loader(self, ds_path: str, shuffle=False):
        #print(self.hparams.cat_names)
        return DataLoader(
            JEFDataset(ds_path, self.hparams.dep_var, self.hparams.text_col, self.hparams.lowercase), 
            batch_size=self.hparams.batch_size,
            shuffle=shuffle,
            pin_memory=True,
            drop_last=True,
            num_workers=int(cpu_count()),
            collate_fn=EncodeCollateFn(self.tokenizer)
        )

    def setup(self, stage):
        if stage == 'fit':
            total_devices = max(1, self.hparams.n_gpus) * 1
            train_batches = len(self.train_dataloader()) // total_devices
            self.train_steps = math.ceil((self.hparams.epochs * train_batches) / self.hparams.accumulate_grad_batches)
            self.train_steps = int(math.ceil(self.train_steps * 1.01))#1.04)

    def configure_optimizers(self):
        train_steps = self.train_steps
        optimizer = torch.optim.AdamW([p for p in self.model.parameters() if p.requires_grad], lr=self.lr, weight_decay=0.1)
        lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                           max_lr=self.lr,
                                                           total_steps=train_steps,
                                                           three_phase=True,
                                                           epochs=self.hparams.epochs)

        return [optimizer], [{"scheduler": lr_scheduler, "interval": "step"}]


Global seed set to 314


In [7]:
learn = TrainingModule.load_from_checkpoint('./bigbird-best.ckpt')

/home/jacob/miniconda3/envs/paper_brcad5/lib/python3.9/site-packages/deprecate/deprecation.py:115: FutureWarning: The `F1` was deprecated since v0.7 in favor of `torchmetrics.classification.f_beta.F1Score`. It will be removed in v0.8.
  stream(template_mgs % msg_args)
Some weights of the model checkpoint at ./bigbird-jus were not used when initializing BigBirdModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'bert.pooler.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that

Freezing all bert encoder
Unfreezing bert encoder last layers
Using weighted loss: 3.8351831436157227


In [8]:
learn.freeze()
learn.eval();

In [9]:
learn.to('cuda:1'); 

In [10]:
learn.device

device(type='cuda', index=1)

In [11]:
learn.hparams

"accumulate_grad_batches": 2
"batch_size":              8
"bert_model_path":         ./bigbird-jus
"bert_unfreeze_mode":      encoder_last
"dep_var":                 label
"deterministic":           True
"device":                  cuda
"drop_mult":               0.3
"epochs":                  15
"gradient_clip_val":       10.0
"hparams":                 Namespace(train_path='../data/train_en.parquet', val_path='../data/valid_en.parquet', test_path='../data/test_en.parquet', batch_size=8, epochs=15, drop_mult=0.3, use_special_classifier='none', lowercase=False, lr=0.0003, thresh_step=0.49, accumulate_grad_batches=2, gradient_clip_val=10.0, stochastic_weight_avg=True, dep_var='label', text_col='preprocessed_full_text_first_instance_court_ruling', bert_model_path='./bigbird-jus', labels=[0, 1], sync_batchnorm=True, device='cuda', train_weights=array(3.835183, dtype=float32), bert_unfreeze_mode='encoder_last', weighted_loss=True, precision='bf16', n_gpus=2, deterministic=True)
"labels":   

In [12]:
def predict_bigbird(model, text):
    input_ids, attention_mask, _ = EncodeCollateFn(learn.tokenizer)([[text, 0]])
    if model.device != 'cpu':
        input_ids = input_ids.to(model.device) 
        attention_mask = attention_mask.to(model.device)
    logits = model.forward(input_ids, attention_mask)
    prob  = torch.sigmoid(logits)
    y_pred = torch.where(prob > model.hparams.thresh_step, 1.0, 0.0).long()
    return logits.cpu().numpy(), prob.cpu().numpy(), int(y_pred)


In [13]:
MODEL_NAME = 'BigBird'
DS_TYPE = 'humans'
DS = df_humans if DS_TYPE == 'humans' else df_test

output_preds = []
bar = progress_bar(DS.preprocessed_full_text_first_instance_court_ruling.values)

for ruling in bar:
    output_preds.append(predict_bigbird(learn, ruling))
    

/home/jacob/miniconda3/envs/paper_brcad5/lib/python3.9/site-packages/transformers/models/big_bird/modeling_big_bird.py:976: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  torch.arange(indices.shape[0] * indices.shape[1] * num_indices_to_gather, device=indices.device)
Attention type 'block_sparse' is not possible if sequence_length: 672 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3.Changing attention type to 'original_full'...


In [14]:
df_preds = pd.Series([i[2] for i in output_preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].replace({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,humans
1,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,humans
2,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,humans
3,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,humans
4,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,humans
...,...,...,...,...
685,PROVIMENTO,PROVIMENTO,BigBird,humans
686,PROVIMENTO,NÃO PROVIMENTO,BigBird,humans
687,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,humans
688,PROVIMENTO,PROVIMENTO,BigBird,humans


In [15]:
MODEL_NAME = 'BigBird'
DS_TYPE = 'test'
DS = df_humans if DS_TYPE == 'humans' else df_test

output_preds = []
bar = progress_bar(DS.preprocessed_full_text_first_instance_court_ruling.values)

for ruling in bar:
    output_preds.append(predict_bigbird(learn, ruling))
    

In [16]:
df_preds = pd.Series([i[2] for i in output_preds], name='y_pred').to_frame()
df_preds['y_true'] = DS.label.values
df_preds['y_pred'] = df_preds['y_pred'].replace({0:'NÃO PROVIMENTO', 1:'PROVIMENTO'})
df_preds['model'] = MODEL_NAME
df_preds['ds_type'] = DS_TYPE
df_preds.to_csv(f'predictions/{MODEL_NAME}-{DS_TYPE}.csv', index=False)
joblib.dump(output_preds, f'predictions/raw-{MODEL_NAME}-{DS_TYPE}.joblib')
df_preds


,y_pred,y_true,model,ds_type
0,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,test
1,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,test
2,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,test
3,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,test
4,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,test
...,...,...,...,...
76294,PROVIMENTO,PROVIMENTO,BigBird,test
76295,PROVIMENTO,PROVIMENTO,BigBird,test
76296,NÃO PROVIMENTO,NÃO PROVIMENTO,BigBird,test
76297,PROVIMENTO,PROVIMENTO,BigBird,test
